In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

### Searching for available experiments

In [2]:
client.search_experiments()

[<Experiment: artifact_location='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/2', creation_time=1685033906002, experiment_id='2', last_update_time=1685033906002, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1', creation_time=1684861233574, experiment_id='1', last_update_time=1684861233574, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1684854511587, experiment_id='0', last_update_time=1684854511587, lifecycle_stage='active', name='Default', tags={}>]

### Create experiment

In [3]:
client.create_experiment("my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active', 1685040125620, 1685040125620)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

### Get the best runs

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.5",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"Run ID: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

Run ID: 035c30e8b8634bc0941ccc5744e9908a, rmse: 6.3103
Run ID: 5bd40bff6e0b4b17b58b603908f36d03, rmse: 6.3108
Run ID: 1ce0fd69d92b46fb8997415be9c478e1, rmse: 6.3157
Run ID: 63a9663bf4ab4e62bce649db5068554f, rmse: 6.3169
Run ID: fbc590c47a0946e49d88ac14c2bdb7bf, rmse: 6.3213


### Interacting with the Model Registry
In this section We will use the MlflowClient instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version 4 was created.
3. Transition the version 4 to "Staging" and adding annotations to it.

In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "e9e4c31e49a24e3189696b1fd31a818f" # This is not the best run, but is the one that contains the preprocessor folder
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2023/05/25 15:42:16 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 6
Created version '6' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685040136656, current_stage='None', description=None, last_updated_timestamp=1685040136656, name='nyc-taxi-regressor', run_id='e9e4c31e49a24e3189696b1fd31a818f', run_link=None, source='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1/e9e4c31e49a24e3189696b1fd31a818f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

### Checking the latest versions of the model
We can check the latest versions of the model and in which stages they are:

In [7]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"Version: {version.version}, stage: {version.current_stage}")

Version: 6, stage: None
Version: 2, stage: Production
Version: 3, stage: Staging


In [8]:
model_version = 4
new_stage = "Staging"

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685038839497, current_stage='Staging', description='The model version 4, was transitioned to Staging, on 2023-05-25', last_updated_timestamp=1685040154934, name='nyc-taxi-regressor', run_id='e9e4c31e49a24e3189696b1fd31a818f', run_link=None, source='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1/e9e4c31e49a24e3189696b1fd31a818f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Updating model version:

In [9]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version}, was transitioned to {new_stage}, on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1685038839497, current_stage='Staging', description='The model version 4, was transitioned to Staging, on 2023-05-25', last_updated_timestamp=1685040165754, name='nyc-taxi-regressor', run_id='e9e4c31e49a24e3189696b1fd31a818f', run_link=None, source='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1/e9e4c31e49a24e3189696b1fd31a818f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

### Comparing versions and selecting the new "Production" model
In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the DictVectorizer that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the DictVectorizer so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [10]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [11]:
df = read_dataframe("../data/green_tripdata_2021-03.parquet")

In [12]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/tmp/ipykernel_26193/2852634549.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/preprocessor'

Loading the preprocessor:

In [13]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [14]:
X_test = preprocess(df, dv)

target = "duration"
y_test = df[target].values

In [15]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

2023/05/25 15:42:58 WARNING mlflow.pyfunc: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - mlflow (current: 2.3.2, required: mlflow==2.3)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


CPU times: user 177 ms, sys: 16.2 ms, total: 193 ms
Wall time: 193 ms


{'rmse': 12.181915014824462}

In [16]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

OSError: No such file or directory: '/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1/e9e4c31e49a24e3189696b1fd31a818f/artifacts/model/.'

In [17]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

<ModelVersion: aliases=[], creation_timestamp=1685038839497, current_stage='Production', description='The model version 4, was transitioned to Staging, on 2023-05-25', last_updated_timestamp=1685040190762, name='nyc-taxi-regressor', run_id='e9e4c31e49a24e3189696b1fd31a818f', run_link=None, source='/home/filipe/Documents/education/mlops-zoomcamp/02-experiment-tracking/theory/mlruns/1/e9e4c31e49a24e3189696b1fd31a818f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>